In [41]:
import gzip
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import nltk
#nltk.download ()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [11]:
import string
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
df = pd.read_json("renttherunway_final_data.json.gz",lines = True)
df = shuffle(df)

In [17]:
total_df = df[["review_text","review_summary","rating","size","fit"]]
total_df["review_total"] = total_df["review_summary"]+ " " +total_df["review_text"]
total_df=total_df.drop(["review_text","review_summary"],axis=1)

/var/folders/kl/8j13qjxs4fd9fz6_ng5b4qpc0000gn/T/ipykernel_17555/201975335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df["review_total"] = total_df["review_summary"]+ " " +total_df["review_text"]


In [18]:
total_df = total_df.dropna(how='any',axis=0) 

In [19]:
def convert_fit(fit):
    if fit == 'large':
        out = 0
    elif fit == 'small':
        out = 1
    else:
        out = 2
    return out
total_df["fit"] = total_df["fit"].apply(lambda x: convert_fit(x))

In [44]:
def convert_rating(rating):
    return rating/2-1
total_df["rating"] = total_df["rating"].apply(lambda x: convert_rating(x))

KeyError: 'rating'

In [21]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
total_df['review_total']= total_df['review_total'].apply(lambda x:remove_punctuation(x))
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens
#applying function to the column
total_df['review_total']= total_df['review_total'].apply(lambda x: tokenization(x))

stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
total_df['review_total']= total_df['review_total'].apply(lambda x:remove_stopwords(x))

wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
total_df['review_total']=total_df['review_total'].apply(lambda x:lemmatizer(x))

In [22]:
def combine(text):
    string = ''
    for i in range(len(text)):
        string += text[i]
        string += ' '
    return string
total_df['review_total']=total_df['review_total'].apply(lambda x:combine(x))

In [24]:
total_df["sentiment_score"] = total_df["rating"]

In [26]:
def convert_review(review):
    sent_dict = sid.polarity_scores(review)
    return sent_dict['compound']
total_df["sentiment_score"] = total_df["review_total"].apply(lambda x: convert_review(x))

In [ ]:
total_df=total_df.drop(["review_total"],axis=1)

In [46]:
label_df = total_df[["rating"]]

In [35]:
total_df=total_df.drop(["rating"],axis=1)

In [47]:
X_train, X_valid,y_train, y_valid = train_test_split(total_df,label_df,test_size=0.2)

In [55]:
xgb = XGBClassifier(max_depth=3,eta=0.3,silent=1,objective='multi:softprob',num_class=3)

In [56]:
model = xgb.fit(X_train, y_train)

[16:12:19] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667849653518/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.



In [62]:
fits = xgb.predict(X_train)
pred = xgb.predict(X_valid)
y_train_arr = y_train["rating"].to_numpy()
y_valid_arr = y_valid["rating"].to_numpy()
acc_xgbfits = (fits == y_train_arr).sum().astype(float) / len(fits)*100
acc_xgb = (pred == y_valid_arr).sum().astype(float) / len(pred)*100

In [64]:
acc_xgb

65.34434832307173

In [65]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_valid)

/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [74]:
acc_knn = (knn_predictions == y_valid_arr).sum().astype(float) / len(knn_predictions)*100
acc_knn

58.795105603616236

In [71]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_valid)

/var/folders/kl/8j13qjxs4fd9fz6_ng5b4qpc0000gn/T/ipykernel_17555/284630940.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)


In [76]:
acc_rf = (rf_predictions == y_valid_arr).sum().astype(float) / len(rf_predictions)*100
acc_rf

61.151378172654766

In [77]:
from sklearn.neural_network import MLPClassifier
snn_classifier = MLPClassifier()
snn_classifier.fit(X_train, y_train)
snn_predictions = snn_classifier.predict(X_valid)

/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [78]:
acc_snn = (snn_predictions == y_valid_arr).sum().astype(float) / len(snn_predictions)*100
acc_snn

65.37552282233133

In [79]:
dnn_classifier = MLPClassifier(hidden_layer_sizes = [100]*5)
dnn_classifier.fit(X_train, y_train)
dnn_predictions = dnn_classifier.predict(X_valid)

/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
acc_dnn = (dnn_predictions == y_valid_arr).sum().astype(float) / len(dnn_predictions)*100
acc_dnn

65.30797807393553

In [81]:
from sklearn.multiclass import OneVsRestClassifier
dnns_classifier = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = [100]*5))
dnns_classifier.fit(np.array(X_train), y_train)
dnns_predictions_labels = dnns_classifier.predict(X_valid)

/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [82]:
acc_dnn = (dnns_predictions_labels == y_valid_arr).sum().astype(float) / len(dnns_predictions_labels)*100
acc_dnn

65.26900994986102

In [ ]:
from sklearn.svm import SVC
svm_classifier = SVC(decision_function_shape='ovr')
svm_classifier.fit(X_train, y_train)
svm_predictions_labels = svm_classifier.predict(X_valid)

/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
acc_svm = (svm_predictions_labels == y_valid_arr).sum().astype(float) / len(svm_predictions_labels)*100
acc_svm

In [ ]:
xgb_classifier = OneVsRestClassifier(XGBClassifier())
xgb_classifier.fit(X_train, y_train)
xbg_predictions_labels = xgb_classifier.predict(X_valid)

In [ ]:
acc_xgb = (xbg_predictions_labels == y_valid_arr).sum().astype(float) / len(xbg_predictions_labels)*100
acc_xgb